In [1]:
import os
from dotenv import load_dotenv
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents import create_agent

load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")


assert GEMINI_API_KEY, "GEMINI_API_KEY is missing. Check your .env file."
assert OPENAI_API_KEY, "OPENAI_API_KEY is missing. Check your .env file."

print("Key loaded:", GEMINI_API_KEY[:6] + "..." )
print("OpenAI Key loaded:", OPENAI_API_KEY[:6] + "...")

# Initialize LLMs
gemini_llm = ChatGoogleGenerativeAI(
    temperature=0,
    model="gemini-2.5-flash",
    google_api_key=GEMINI_API_KEY
)
openai_llm = ChatOpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    openai_api_key=OPENAI_API_KEY
)

# Prompt template
prompt = PromptTemplate(
    input_variables=["product"],
    template="Give me a creative name for a company that makes {product}?",
)


# Example: Use OpenAI
chain = prompt | openai_llm
response = chain.invoke({"product": "smart home devices"})
print("OpenAI response:", response)

Key loaded: AIzaSy...
OpenAI Key loaded: sk-pro...
OpenAI response: content='"IntelliHome Innovations"' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 21, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-CrNcFruRd9mx2srPc7qDmN54jb1A8', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='lc_run--019b5fc3-62e8-71a0-888f-c3d8da1dfdd0-0' usage_metadata={'input_tokens': 21, 'output_tokens': 7, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}


In [4]:
# === Cell 2: short‑term memory tuning + enterprise-style multi-user chat ===
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent, AgentState
from langchain.agents.middleware import before_model
from langgraph.checkpoint.memory import InMemorySaver

# 1) Simple "tuning": keep only the last 5 messages
@before_model
def trim_messages(state: AgentState, _runtime):
    msgs = state["messages"]
    if len(msgs) <= 5:
        return None
    return {"messages": msgs[-5:]}

# 2) Agent with short-term memory
agent = create_agent(
    model=ChatOpenAI(model="gpt-4o-mini"),
    tools=[],
    state_schema=AgentState,
    middleware=[trim_messages],
    checkpointer=InMemorySaver(),
)

# 3) Enterprise-style helper: multi-user, multi-thread
def ask(user_id: str, thread_id: str, text: str) -> str:
    config = {"configurable": {"thread_id": f"{user_id}:{thread_id}"}}
    result = agent.invoke(
        {"messages": [{"role": "user", "content": text}]},
        config,
    )
    return result["messages"][-1].content

# Examples
print("U1-T1:", ask("user_1", "ticket_001", "Hi! My name is Alice. Remember it."))
print("U1-T1:", ask("user_1", "ticket_001", "What is my name?"))

print("U2-T2:", ask("user_2", "ticket_002", "Hi! My name is Bob. Remember it."))
print("U2-T2:", ask("user_2", "ticket_002", "What is my name?"))

U1-T1: Hi, Alice! I can remember our conversation while we're chatting, but I won't be able to retain your name in future interactions. How can I assist you today?
U1-T1: Your name is Alice! How can I help you today?
U2-T2: Hi Bob! I can remember your name for this conversation. How can I assist you today?
U2-T2: Your name is Bob! How can I help you today?
